In [1]:
import pymongo
from pymongo import MongoClient
from bson.son import SON
from bson.objectid import ObjectId
import rdflib

In [2]:
client = MongoClient()
db = client.lobbyradar

Entities = db.entities
Relations = db.relations

person_filter = { 'type' : 'person' }
entity_filter = { 'type' : 'entity' }
persons = Entities.find(person_filter)
entities = Entities.find(entity_filter)

In [5]:
# executive, ececutive, Vorsitzender
directors_board_members = ['Aufsichtsratsmitglied'', ''Board of Directors'', ''Mitglied des Aufsichtsrats'', ''Mitglied im Aufsichtsrat'', ''Lenkungsgruppe'', ''Mitglied des Aufsichtsrates'', ''Anteilseignervertreter'', ''Stellvertretender Vorstandsvorsitzender des Aufsichtsrats']
management_board_members = ['Vorstandsmitglied'', ''Mitglied des Vorstands']
presidial_board_members = ['Präsidialmitglied', 'Mitglied des Präsidiums', 'Präsidiumsmitglied']
chairmen_of_management = ['Vorstandsvorsitz', 'Vorstandsvorsitzender', 'Vorsitzender des Vorstands']
management_members = ['Vorstand']
chairmen_of_directors_board = ['Aufsichtsratsvorsitz', 'Vorsitzender des Aufsichtsrats']
foundation_board_members = ['Mitglied des Stiftungsrats', 'Stiftungsrat']
directors = ['Geschäftsführer', 'Hauptgeschäftsführer', 'CEO', 'Geschäftsführender Vorstand', 'Geschäftsführender Gesellschafter', 'Direktor']
presidents = ['Präsident', 'Vorsitz', 'Vorsitzender', 'Präsidentin']
trustees_board_members = ['Mitglied des Kuratoriums']
advidory_board_members = ['Mitglied im Beirat', 'Mitglied des Beirats', 'Beisitzer des Beirats']
vice_presidents = ['Stellvertretender Vorsitz', 'Vize-Präsident', 'Vizepräsident']
other_executive_members = ['Mitglied des Verwaltungsrats']

In [6]:
# member, activity, position
ordinary_members = ['Ordentliches Mitglied']
representative_members = ['Stellvertretendes Mitglied']
chairmen = ['Obmann']
chairwomen = ['Obfrau']
alliance_members = ['Mitglied im Verband']
honorary_members = ['Ehrenmitglied']

In [ ]:
# government, Bundesdatenschutzbeauftragte
ministers = ['Staatsminister', 'Staatsministerin', 'Bundesminister für Umwelt, Naturschutz und Reaktorsicherheit', 'Bundesministerin für Familie, Senioren, Frauen und Jugend', 'Bundesminister für Wirtschaft und Technologie', 'Minister für Inneres und Kommunales', 'Minister für Inneres und Sport', 'Bundesminister für Ernährung, Landwirtschaft und Verbraucherschutz', 'Bundesministerin für Bildung, Wissenschaft, Forschung und Technologie', 'Bundesminister für besondere Aufgaben', 'Bundesminister für Arbeit und Soziales', 'Bundesminister des Auswärtigen', 'Bundesminister des Inneren', 'Minister für Umwelt und Verbraucherschutz', 'Bundesminister für Verbraucherschutz', 'Ernährung und Landwirtschaft', 'Minister der Justiz und für Verbraucherschutz', 'Bundesminister für Verkehr, Bau und Stadtentwicklung', 'Staatsrätin für Bundesangelegenheiten, Europa und Integration', 'Ministerin für Arbeit, Gleichstellung und Soziales', 'Minister für Energie, Infrastruktur und Landesentwicklung', 'Ministerin für Kultus', 'Ministerin für Schule und Berufsbildung', 'Senator für Inneres und Sport', 'Ministerin für Bildung, Wissenschaft, Weiterbildung und Kultur', 'Minister für Wirtschaft, Arbeit und Verkehr', 'Ministerin für Arbeit, Soziales, Gesundheit, Frauen und Familie']
secretaries_of_states = ['Staatssekretär', 'Parlamentarischer Staatssekretär', 'Parlamentarische Staatssekretärin', 'Staatssekretärin']
privy_counselors = ['Staatsrat', 'Staatsrätin']
prime_ministers = ['Ministerpräsident', 'Ministerpräsidentin']
heads_of_state_chancellery = ['Chef der Staatskanzlei']
representatives_of_federal_chancellor = ['Stellvertreter der Bundeskanzlerin']
ministers_of_defense = ['Bundesminister der Verteidigung']
representatives_of_prime_minister = ['Stellvertretender Ministerpräsident', 'Stellvertreter des Ministerpräsidenten', 'Stellvertretende Ministerpräsidentin']
finance_ministers = ['Bundesminister der Finanzen', 'Finanzminister', 'Minister der Finanzen ']
justice_ministers = ['Minister der Justiz', 'Bundesministerin der Justiz', 'Justizministerin']

In [ ]:
import rdflib
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF, SKOS
ORG = Namespace("http://www.w3.org/ns/org#")
OWN = Namespace("http://example.com/")

# The Graph
g = Graph()
g.bind("dc", DC)
g.bind("foaf", FOAF)
g.bind("org", ORG)
g.bind("skos", SKOS)

g.parse('ontology.ttl', format='turtle')

for entity in Entities.find({}):
    node = BNode()
    # DC
    g.add((node, DC.identifier, Literal(entity["_id"])))
    g.add((node, DC.created, Literal(entity["created"])))
    g.add((node, DC.modified, Literal(entity["updated"])))
    # RDF
    if entity['type'] == 'person':
        g.add((node, RDF.type, FOAF.Person))
        g.add((node, FOAF.name, Literal(entity["name"])))

    elif entity['type'] == 'entity':
        g.add((node, RDF.type, ORG.Organization))
        g.add((node, SKOS.prefLabel, Literal(entity["name"])))
        for alias in entity["aliases"]:
            g.add((node, SKOS.altLabel, Literal(alias)))
    # FOAF
    for tag in entity['tags']:
        g.add((node, FOAF.topic_intest, Literal(tag)))